In [1]:
import numpy as np
import gym
from collections import defaultdict

In [2]:
env = gym.make('Blackjack-v0')

/home/ninja/.local/lib/python3.6/site-packages/gym/envs/registration.py:14: PkgResourcesDeprecationWarning: Parameters to load are deprecated.  Call .resolve and .require separately.
  result = entry_point.load(False)


In [5]:
'''
Ao reiniciar o ambiente recebemos o estado inicial:
(
  soma das cartas na mão do jogador,
  soma das cartas na mão do dealer,
  ás usável
  )
'''
state = env.reset()
state

In [128]:
'''
Há duas ações possíveis:
0 - stick, não pedir carta
1 - hit, pedir mais uma carta
'''
env.action_space

Discrete(2)

In [8]:
'''
Ao escolher uma ação recebemos:
o novo estado,
a recompensa e
um booleano: (True = fim de jogo / False = jogo continua)
'''
next_state = env.step(1)
next_state

In [7]:
'''
As possíveis combinações retornadas por env.step()
formam o espaço de observação
'''
env.observation_space

Tuple(Discrete(32), Discrete(11), Discrete(2))

In [11]:
'''
A estratégia é pedir mais uma carta se a soma da mão for 
menor que 20 e caso contrário parar.
'''
def policy(state):
    hand, dealer_hand, usable_ace = state
    return 0 if hand >=20 else 1

In [12]:
'''
Geramos um episódio inteiro usando a política definida
'''
def get_episode():
    state = env.reset()
    episode = []
    while True:
        action = policy(state)
        next_state, reward, done, _ = env.step(action)
        episode.append((state,reward,action))
        if done: break
        state = next_state
    return episode

In [243]:
'''
Aplicamos o método de monte carlo para fazer a previsão 
dos valores dos esdados
'''
def bj_play(n_episodes = 100,verbose=1):
    returns = defaultdict(float)
    num_states = defaultdict(float)
    V = defaultdict(float)
    R = []
    for i in range(n_episodes):
        G = 0
        episode = get_episode()
        states = [item[0] for item in episode]
        rewards = [item[1] for item in episode]
        action = [item[2] for item in episode]
        R.append(sum(rewards))
            
        for t in range(len(episode)):
            G = 0.95*G + rewards[t]
            returns[states[t]] += G
            num_states[states[t]] += 1
            V[states[t]] = V[states[t]]+ 1/num_states[states[t]]*(G-V[states[t]])
    
    if verbose ==1:
        r_pos = sum([i for i in R if i>0])
        r_draw = sum([1 for i in R if i==0])
        r_neg = sum([i for i in R if i<0])
        p_win = r_pos/(r_pos + abs(r_neg)+r_draw)
        p_lose = abs(r_neg)/(r_pos + abs(r_neg)+r_draw)
        p_draw = r_draw/(r_pos + abs(r_neg)+r_draw)
        print('Porcentagem de vitórias: {:.2%}\nPorcentagem de derrotas: {:.2%}\nPorcentagem de empates: {:.2%}'.format(p_win,p_lose,p_draw))
        print('')
        print('Retona: State,State_value.')
    
    return V

In [244]:
%%time
V = bj_play(100000,1)

Porcentagem de vitórias: 29.22%
Porcentagem de derrotas: 64.92%
Porcentagem de empates: 5.86%

Retona: State,State_value.
CPU times: user 4.72 s, sys: 0 ns, total: 4.72 s
Wall time: 4.72 s


In [238]:
'''
Geramos 250 estados diferentes
'''
len(set(V))

280

In [239]:
list(V.items())[:10]

[((21, 8, True), 0.9343544857768049),
 ((21, 3, True), 0.8596112311015123),
 ((21, 10, True), 0.8917333333333335),
 ((16, 10, False), -0.6099534199558697),
 ((15, 10, True), 0.0),
 ((13, 10, False), -0.38617245005257456),
 ((20, 10, False), 0.4306930693069298),
 ((18, 5, False), -0.7707762557077623),
 ((8, 8, False), 0.0),
 ((11, 8, False), 0.0)]

In [240]:
sorted(V.items(),reverse=True)[:10]

[((21, 10, True), 0.8917333333333335),
 ((21, 10, False), 0.8822531387852054),
 ((21, 9, True), 0.9434889434889432),
 ((21, 9, False), 0.9363143631436311),
 ((21, 8, True), 0.9343544857768049),
 ((21, 8, False), 0.933852140077821),
 ((21, 7, True), 0.882121807465619),
 ((21, 7, False), 0.9197930142302717),
 ((21, 6, True), 0.8987068965517251),
 ((21, 6, False), 0.8799999999999998)]